In [2]:
import torch
print(torch.cuda.is_available())         # should return True
print(torch.cuda.get_device_name(0))     # should say "NVIDIA GeForce RTX 3060"


True
NVIDIA GeForce RTX 3060


Imports 

In [28]:
import os

# For data manipulation and analysis 
import pandas as pd

# For numerical operations and array handling 
import numpy as np

# For creating plots and visualisations
import matplotlib.pylab as plt

# For advanced data visualisations 
import seaborn as sns

# For file pattern matching 
import glob as glob

# For audio processing 
import librosa

# For displaying audio data visually 
import librosa.display

# For playing audio directly in notebooks 
import IPython.display as ipd

import wave

#for reproduability 
import random

#for the fodler re-structure 150 sample 
import shutil





Loading dataset 

In [29]:
base_dir = os.path.join(os.getcwd(), 'datasets', 'voice-based-id-recognition')
diffPhrase = os.path.join(base_dir,'differentPhrase')
samePhrase = os.path.join(base_dir,'samePhrase')

Checking Dataset Location existance

In [30]:
# Check if the training directory exists
if os.path.exists(diffPhrase):
    print("differentPhrase Directory: ")
    print(os.listdir(diffPhrase))
else:
    print("Directory does not exist:", diffPhrase) 
   

# Check if the testing directory exists
if os.path.exists(samePhrase):
    
    print("samePhrase Directory: ")
    print(os.listdir(samePhrase))
else:
    print("Directory does not exist:", samePhrase)

differentPhrase Directory: 
['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
samePhrase Directory: 
['1', 

Listen to audio files

In [31]:
"""

file_path = r"C:\Users\Aadil\Desktop\Voice-Auth-DF-System\ml-models\datasets\voice-based-id-recognition\differentPhrase\1\1-11.wav"

with wave.open(file_path, 'rb') as wav_file:
    num_frames = wav_file.getnframes()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()
    channels = wav_file.getnchannels()
    audio_data = wav_file.readframes(num_frames)

# Convert raw bytes to numpy array
audio_np = np.frombuffer(audio_data, dtype=np.int16)

# If stereo, reshape
if channels == 2:
    audio_np = audio_np.reshape(-1, 2)

ipd.Audio(audio_np, rate=sample_rate)


"""


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 18-19: truncated \UXXXXXXXX escape (3963867248.py, line 22)

Pre-Processing 

In [ ]:
#Dataset split
base_path = os.path.join(os.getcwd(), 'datasets', 'vox1_dev')
print(base_path)

#Each folder contains a different speaker

speaker_count = sum(os.path.isdir(os.path.join(base_path, entry)) for entry in os.listdir(base_path))
print("Number of original Speakers : ",speaker_count)
#Select a random selection of speakers, as inital dataset of 1211 speakers is roughly 30GB. will take a sample of 150 speakers(12.3%), aroudn 3.7GB

""""
# Destination path for subset
train_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')
os.makedirs(train_path, exist_ok=True)


# Set seed and select random 150 speakers
random.seed(42)
all_speakers = [entry for entry in os.listdir(base_path)
                if os.path.isdir(os.path.join(base_path, entry))]
selected_speakers = random.sample(all_speakers, 150)

# Copy selected speakers and their full contents to the subset folder
for speaker in selected_speakers:
    src_speaker_path = os.path.join(base_path, speaker)
    dest_speaker_path = os.path.join(train_path, speaker)
    shutil.copytree(src_speaker_path, dest_speaker_path)

#print(f"Subset dataset created at: {train_path}")
"""

# Print the selected speaker folders
count = 0
#print("Selected speakers:")
for speaker in selected_speakers:
    count +=1
    #print(speaker)

print("Number of train Speakers : ",count)



test_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_test')

test_speaker_count = sum(os.path.isdir(os.path.join(test_path, entry)) for entry in os.listdir(test_path))
print("Number of test Speakers : ",test_speaker_count)

c:\Users\Aadil\Desktop\Voice-Auth-DF-System\ml-models\datasets\vox1_dev
Number of original Speakers :  1211
Number of train Speakers :  150
Number of test Speakers :  40


testing to see if files are copied in both train and test, tested using hash of the files to compare. 

In [ ]:
import os
import hashlib

def file_hash(filepath):
    hasher = hashlib.sha256()
    with open(filepath, 'rb') as f:
        hasher.update(f.read())
    return hasher.hexdigest()

# Define your paths
test_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_test')
train_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')

# Map filenames to full paths
def get_file_map(base_path):
    file_map = {}
    for root, _, files in os.walk(base_path):
        for file in files:
            file_map[file] = os.path.join(root, file)
    return file_map

test_files = get_file_map(test_path)
train_files = get_file_map(train_path)

# Compare common files
common = set(test_files).intersection(train_files)
print(f"Found {len(common)} common files.\n")

for filename in sorted(common):
    test_file = test_files[filename]
    train_file = train_files[filename]

    same = file_hash(test_file) == file_hash(train_file)
    print(f"{filename} - {'Same' if same else 'Different'}")



Found 55 common files.

00001.wav - Different
00002.wav - Different
00003.wav - Different
00004.wav - Different
00005.wav - Different
00006.wav - Different
00007.wav - Different
00008.wav - Different
00009.wav - Different
00010.wav - Different
00011.wav - Different
00012.wav - Different
00013.wav - Different
00014.wav - Different
00015.wav - Different
00016.wav - Different
00017.wav - Different
00018.wav - Different
00019.wav - Different
00020.wav - Different
00021.wav - Different
00022.wav - Different
00023.wav - Different
00024.wav - Different
00025.wav - Different
00026.wav - Different
00027.wav - Different
00028.wav - Different
00029.wav - Different
00030.wav - Different
00031.wav - Different
00032.wav - Different
00033.wav - Different
00034.wav - Different
00035.wav - Different
00036.wav - Different
00037.wav - Different
00038.wav - Different
00039.wav - Different
00040.wav - Different
00041.wav - Different
00042.wav - Different
00043.wav - Different
00044.wav - Different
00045.wa

Spliting train and validaion(80/20 split)

In [ ]:

# Path to the dataset with 150 speaker folders
source_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')

# Output subfolders for train and val
train_folder = os.path.join(source_path, 'train_data')
val_folder = os.path.join(source_path, 'val_data')

# Create output folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Get all speaker folders (exclude train_data and val_data if rerunning)
speaker_folders = [folder for folder in os.listdir(source_path)
                   if os.path.isdir(os.path.join(source_path, folder)) and folder not in ['train_data', 'val_data']]

# Shuffle and split 80/20
random.seed(42)
random.shuffle(speaker_folders)
split_index = int(0.8 * len(speaker_folders))
train_speakers = speaker_folders[:split_index]
val_speakers = speaker_folders[split_index:]

# Copy folders to train_data
for speaker in train_speakers:
    src = os.path.join(source_path, speaker)
    dst = os.path.join(train_folder, speaker)
    shutil.move(src, dst)

# Copy folders to val_data
for speaker in val_speakers:
    src = os.path.join(source_path, speaker)
    dst = os.path.join(val_folder, speaker)
    shutil.move(src, dst)

print(f"{len(train_speakers)} speakers moved to train_data, {len(val_speakers)} to val_data.")



Done! 120 speakers moved to train_data, 30 to val_data.
